In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/dataco-smart-supply-chain-for-big-data-analysis/DataCoSupplyChainDataset.csv', encoding='ISO-8859-1')
df.head()

## Data cleaning

In [ ]:
np.sum(df.isna())

In [ ]:
# Replace missing values in 'Customer Zipcode' and 'Order Zipcode' with 0
df['Customer Zipcode'] = df['Customer Zipcode'].fillna(0)
df['Order Zipcode'] = df['Order Zipcode'].fillna(0)

# Replace missing values in 'Customer Fname' with "NotDetermined"
df['Customer Fname'] = df['Customer Fname'].fillna("NotDetermined")

# Create 'Customer Full Name' by combining 'Customer Fname' and 'Customer Lname'
df['Customer Full Name'] = df['Customer Fname'].astype(str) + df['Customer Lname'].astype(str)

In [ ]:
dfData=df.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname','Customer Lname',
           'Product Description','Product Image','Order Zipcode','Customer Zipcode'],axis=1)
dfData.shape

In [ ]:
dfData.columns = [col.lower().replace(' ', '_') for col in dfData.columns]
dfData.rename(columns=lambda x: x.replace("(", "").replace(")", ""), inplace=True)
dfData.columns

In [ ]:
Delivery=dfData.groupby('delivery_status')
Market = dfData.groupby('market') 
Region = dfData.groupby('order_region')
Customer=dfData.groupby('customer_segment')
Category=dfData.groupby('category_name')
Shipping=dfData.groupby('shipping_mode')
Region=dfData.groupby('order_region')

## Customer Segment Analysis

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=dfData, x='sales', bins=30, kde=True)
plt.title('Distribution of Sales')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.show()

In [ ]:
customer_segment_counts = dfData['customer_segment'].value_counts()

In [ ]:
# Plotting Customer Segment Counts
plt.figure(figsize=(10, 6))
sns.barplot(x=customer_segment_counts.index, y=customer_segment_counts.values)
plt.title('Customer Segment Distribution')
plt.xlabel('Customer Segment')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(customer_segment_counts, labels=customer_segment_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Customer Segments',loc='left', pad=20)
plt.axis('equal')
plt.legend(customer_segment_counts.index, title='Customer Segments', loc='best')
plt.show()

In all the preceding charts, the consumer category consistently stands out as the largest segment. Consequently, it becomes imperative to emphasize precise demand forecasting and punctual delivery for this particular customer group.

## Market Analysis

In [ ]:
market_counts = dfData['market'].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=market_counts.index, y=market_counts.values)
plt.title('Market Analysis - Distribution by Market')
plt.xlabel('Market')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(market_counts, labels=market_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Market Analysis - Distribution by Market', loc='left', pad=20)
plt.axis('equal') 
plt.legend(market_counts.index, title='Market', loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(10,12))
plt.subplot(6, 2, 1)
Category['sales'].sum().sort_values(ascending=False).plot.bar(figsize=(18,18), title="Total Sales")

plt.subplot(6, 2, 2)
Category['order_item_quantity'].sum().sort_values(ascending=False).plot.bar(figsize=(18,18), title="Total Quantity")

plt.subplot(6, 2, 3)
Category['sales_per_customer'].mean().sort_values(ascending=False).plot.bar(figsize=(18,18), title="Average sales")

plt.subplot(6, 2, 4)
Category['product_price'].mean().sort_values(ascending=False).plot.bar(figsize=(18,18), title="Average price")


plt.tight_layout()
plt.show()

## Risk Management - Delivery 

In [ ]:
delivery_status_data = dfData['delivery_status']

# Summary Statistics
delivery_status_summary = delivery_status_data.value_counts()

# Visualization
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")
sns.countplot(data=dfData, x='delivery_status', order=delivery_status_summary.index, palette='Set2')
plt.title('Delivery Status Distribution')
plt.xlabel('Delivery Status')
plt.ylabel('Count')

# Display summary statistics
print("Delivery Status Summary:")
print(delivery_status_summary)

In [ ]:
late_deliveries = dfData[dfData['delivery_status'] == 'Late delivery']

In [ ]:
delivery_status_data = dfData['delivery_status']
shipping_mode_data = dfData['shipping_mode']

# Create a cross-tabulation (contingency table) to analyze the relationship
cross_tab = pd.crosstab(shipping_mode_data, delivery_status_data)

# Calculate row percentages
cross_tab_percent = cross_tab.div(cross_tab.sum(1), axis=0) * 100

# Visualize the cross-tabulation as a stacked bar chart with percentages
plt.figure(figsize=(12, 8))
sns.set(style="whitegrid")
cross_tab_percent.plot(kind="bar", stacked=True, colormap='Set2')
plt.title('Delivery Status by Shipping Mode (Percentage)')
plt.xlabel('Shipping Mode')
plt.ylabel('Percentage')

# Display the cross-tabulation with percentages
print("Cross-Tabulation (Contingency Table) with Percentages:")
print(cross_tab_percent)

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
dfData['late_delivery_risk'] = dfData['late_delivery_risk'].map({0: 'no late delivery', 1: 'late delivery'})

palette = {'no late delivery': 'green', 'late delivery': 'red'}

scatter = sns.scatterplot(x="days_for_shipment_scheduled", y="days_for_shipping_real", hue="late_delivery_risk", data=dfData, palette=palette, s=80)
plt.title('Scatterplot of Days for Shipment (Scheduled vs. Actual)')
plt.xlabel('Days for Shipment (Scheduled)')
plt.ylabel('Days for Shipment (Actual)')

plt.grid(True, linestyle='--', alpha=0.7)

plt.show()

According to the chart, the orders that were scheduled to be delivered in 2 days have the highest delay. By analyzing the dataset, 2-day deliveries are related to second-class shipping modes. As a result, the highest amount of delay in delivery is related to the Second Class. Shipping Modes (Days for shipment scheduled):

Standard Class (4 Days) Second Class (2 Days) First Class (1 Day) Same Day (0 Day)

## Classification Models for Late Delivery

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_Data = dfData.copy()

In [ ]:
train_Data['late_delivery']=np.where(train_Data['delivery_status'] == 'Late delivery', 1, 0) 

In [ ]:
train_Data.head()

In [ ]:
train_data=train_Data.drop(['delivery_status','late_delivery_risk'], axis=1) #Dropping columns with repeated values

In [ ]:
train_data.head()

In [ ]:
train_data.columns

In [ ]:

# List of columns to encode
columns_to_encode = [
    'customer_country', 'market', 'type', 'product_name', 'customer_segment',
    'customer_state', 'order_region', 'order_city', 'category_name',
    'customer_city', 'department_name', 'order_state', 'order_status',
    'shipping_mode', 'order_country', 'customer_full_name'
]

# Initialize the LabelEncoder
le = LabelEncoder()

# Encode categorical columns
for col in columns_to_encode:
    train_data[col] = le.fit_transform(train_data[col])

In [ ]:
data=train_data.drop(['shipping_date_dateorders','order_date_dateorders'],axis=1)

In [ ]:
xlatedelivery=data .loc[:, data .columns != 'late_delivery']
ylatedelivery=data['late_delivery']
xlatedelivery_train, xlatedelivery_test,ylatedelivery_train,ylatedelivery_test = train_test_split(xlatedelivery,ylatedelivery,test_size = 0.3, random_state = 42)

In [ ]:
scaler = StandardScaler()
xlatedelivery_train = scaler.fit_transform(xlatedelivery_train)
xlatedelivery_test = scaler.transform(xlatedelivery_test)

In [ ]:
def classifiermodel(model_latedelivery, xlatedelivery_train, xlatedelivery_test, ylatedelivery_train, ylatedelivery_test):
    model_latedelivery = model_latedelivery.fit(xlatedelivery_train, ylatedelivery_train)
    ylatedelivery_pred = model_latedelivery.predict(xlatedelivery_test)
    accuracy_latedelivery = accuracy_score(ylatedelivery_pred, ylatedelivery_test)
    recall_latedelivery = recall_score(ylatedelivery_pred, ylatedelivery_test)
    conf_latedelivery = confusion_matrix(ylatedelivery_test, ylatedelivery_pred)
    f1_latedelivery = f1_score(ylatedelivery_test, ylatedelivery_pred)
    print('Model paramters used are:', model_latedelivery)
    print('Accuracy of late delivery status is:', accuracy_latedelivery * 100, '%')
    print('Recall score of late delivery status is:', recall_latedelivery * 100, '%')
    print('F1 score of late delivery status is:', f1_latedelivery * 100, '%')
    print('Conf Matrix of late delivery status is:\n', conf_latedelivery)

In [ ]:
# Initialize and train the models
model_latedelivery_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
classifiermodel(model_latedelivery_rf, xlatedelivery_train, xlatedelivery_test, ylatedelivery_train, ylatedelivery_test)

model_latedelivery_svm = LinearSVC()
classifiermodel(model_latedelivery_svm, xlatedelivery_train, xlatedelivery_test, ylatedelivery_train, ylatedelivery_test)

model_latedelivery_lr = LogisticRegression(solver='lbfgs', random_state=0)
classifiermodel(model_latedelivery_lr, xlatedelivery_train, xlatedelivery_test, ylatedelivery_train, ylatedelivery_test)

model_latedelivery_lda = LinearDiscriminantAnalysis()
classifiermodel(model_latedelivery_lda, xlatedelivery_train, xlatedelivery_test, ylatedelivery_train, ylatedelivery_test)

model_latedelivery_nb = GaussianNB()
classifiermodel(model_latedelivery_nb, xlatedelivery_train, xlatedelivery_test, ylatedelivery_train, ylatedelivery_test)

In [ ]:

# Create a DataFrame to compare classification results
Classification_data = {
    'Classification Model': ['Random Forest Classification', 'Support Vector Machines', 'Logistic Classification Model',
                             'Linear Discriminant Analysis', 'Gaussian Naive Bayes Model'],
    'Accuracy': [99.44419824211536, 98.25688751015585, 98.24950144028362, 96.20540660314647, 94.58416426619397],
    'Recall': [98.99820275577447, 96.93315516735652, 96.93275530493172, 96.25793518209154, 91.21879227647275],
    'F1': [99.4965797528056, 98.4378102866221, 98.43108698530386, 96.55634687892753, 95.28979106778654],
    'TN': [24110, 23470, 23470, 23291, 21555],
    'FP': [301, 941, 941, 1120, 2856],
    'FN': [0, 3, 7, 935, 77],
    'TP': [29745, 29742, 29738, 28810, 29668]
}

Classification_comparision = pd.DataFrame(Classification_data, columns=['Classification Model', 'Accuracy', 'Recall', 'F1', 'TN', 'FP', 'FN', 'TP'])

Classification_comparision

Random forest classifier is performing better for classification type, with an F1 score of 99.49% for late delivery.